# Многослойные сети с `Flux.jl`

Ранее мы видели, что одного слоя нейронов недостаточно, чтобы различать три типа объектов (в нашем случае: яблоки, бананы и виноград), поскольку данные довольно сложны. Чтобы решить эту проблему, нам нужно использовать больше слоев, поэтому отправляйтесь на территорию **глубокого обучения**! 

Добавив еще один слой между входными и выходными нейронами, так называемый «скрытый слой», мы получим нашу первую серьезную **нейронную сеть**, которая выглядит примерно так:

In [ ]:
include("draw_neural_net.jl")
draw_network([2, 4, 3])

Мы продолжим использовать два входных параметра и попробуем классифицировать их на три типа, поэтому у нас будет три выходных нейрона. Мы решили добавить один «скрытый слой» с 4 нейронами. 

Большая часть «искусства» глубокого обучения - это выбор подходящей структуры для нейронной сети, которая позволит модели быть достаточно сложной, чтобы хорошо моделировать данные, но достаточно простой, чтобы параметры могли быть изучены в разумных пределах.

## Считывание и обработка данных

Как и прежде, давайте загрузим некоторые предварительно обработанные данные, используя код, который мы видели в предыдущем блокноте.

In [ ]:
using Flux
using Flux: onehot
using CSV, TextParse, DataFrames

In [ ]:
#=
apples_1 = CSV.read("data/Apple_Golden_1.dat", delim='\t')
apples_2 = CSV.read("data/Apple_Golden_2.dat", delim='\t')
apples_3 = CSV.read("data/Apple_Golden_3.dat", delim='\t')
bananas = CSV.read("data/Banana.dat", delim='\t')
grapes_1 = CSV.read("data/Grape_White.dat", delim='\t')
grapes_2 = CSV.read("data/Grape_White_2.dat", delim='\t');
=#

applecols1, applecolnames1 = TextParse.csvread("data/Apple_Golden_1.dat", '\t')
applecols2, applecolnames2 = TextParse.csvread("data/Apple_Golden_2.dat", '\t')
applecols3, applecolnames3 = TextParse.csvread("data/Apple_Golden_3.dat", '\t')
bananacols, bananacolnames = TextParse.csvread("data/Banana.dat", '\t');
grapecols1, grapecolnames1 = TextParse.csvread("data/Grape_White.dat", '\t');
grapecols2, grapecolnames2 = TextParse.csvread("data/Grape_White_2.dat", '\t');

apples_1 = DataFrame([col for col in applecols1], [Meta.parse(ac) for ac in applecolnames1])
apples_2 = DataFrame([col for col in applecols2], [Meta.parse(ac) for ac in applecolnames2])
apples_3 = DataFrame([col for col in applecols3], [Meta.parse(ac) for ac in applecolnames3])
bananas  = DataFrame([col for col in bananacols], [Meta.parse(bc) for bc in bananacolnames])
grapes_1 = DataFrame([col for col in grapecols1], [Meta.parse(bc) for bc in grapecolnames1])
grapes_2 = DataFrame([col for col in grapecols2], [Meta.parse(bc) for bc in grapecolnames2])

apples = vcat(apples_1, apples_2, apples_3)
grapes = vcat(grapes_1, grapes_2);

In [ ]:
col1 = :red
col2 = :blue

x_apples  = [ [apples_1[i, col1], apples_1[i, col2]] for i in 1:size(apples_1)[1] ]
append!(x_apples, [ [apples_2[i, col1], apples_2[i, col2]] for i in 1:size(apples_2)[1] ])
append!(x_apples, [ [apples_3[i, col1], apples_3[i, col2]] for i in 1:size(apples_3)[1] ])

x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

x_grapes = [ [grapes_1[i, col1], grapes_1[i, col2]] for i in 1:size(grapes_1)[1] ]
append!(x_grapes, [ [grapes_2[i, col1], grapes_2[i, col2]] for i in 1:size(grapes_2)[1] ])

xs = vcat(x_apples, x_bananas, x_grapes);

Теперь мы хотим классифицировать три типа фруктов, поэтому мы снова используем one-hot векторы для представления желаемых результатов $y^{(i)}$:

In [ ]:
labels = [ones(length(x_apples)); 2*ones(length(x_bananas)); 3*ones(length(x_grapes))];

ys = [onehot(label, 1:3) for label in labels];  # onehotbatch(labels, 1:3)

Входные данные находятся в `xs`, а векторы ответы - в` ys`.

## Несколько слоев в Flux

Давайте расскажем Flux, какую структуру мы хотим иметь в сети. Сначала мы указываем количество нейронов в каждом слое, а затем создаем каждый слой как «плотный» слой:

In [ ]:
inputs = 2
hidden = 4
outputs = 3

layer1 = Dense(inputs, hidden, σ)
layer2 = Dense(hidden, outputs, σ)

Чтобы объединить несколько слоев в многослойную сеть, мы используем функцию `Chain` от Flux:

In [ ]:
model = Chain(layer1, layer2)

#### Упражнение 1

Какова внутренняя структура и подструктура этого «модельного» объекта?

## Обучение модели

Сейчас мы создали модель, и у нас есть данные обучения. Как мы обучаем модель на данных? 

Удивительно, что остальная часть кода в `Flux` **точно такая же, как и раньше**. Это возможно благодаря дизайну самой Юлии и пакета «Flux».

#### Упражнение 2

Тренируйте модель, как и раньше, теперь с помощью популярного оптимизатора ADAM. Возможно, вам придется обучать сеть дольше, чем раньше, поскольку у нас гораздо больше параметров.

## Визуализация результатов

Что представляет собой эта нейронная сеть? Это просто более сложная функция с двумя входами и тремя выходами, то есть функция $ f: \mathbb {R} ^ 2 \to \mathbb {R} ^ 3 $. 

Раньше с одним слоем каждый компонент функции $ f $ в основном соответствовал гиперплоскости; теперь это будет **более сложная нелинейная функция** входных данных!

#### Упражнение 3

Визуализируйте каждый компонент выходных данных отдельно в виде тепловой карты и / или контуров, наложенных на данные. Интерпретировать результаты.

## Что мы узнали

Добавление промежуточного слоя позволяет сети начать деформировать разделяющие поверхности, которые она изучает, в более сложные, нелинейные (изогнутые) формы. Это позволяет отделить данные, которые ранее не могли быть разделены!

Однако использование только двух функций означает, что данные из разных классов перекрываются. Чтобы отличить его, нам нужно использовать больше возможностей.

### Упражнение 4

Используйте три признака (красный, зеленый и синий) и создайте сеть с одним скрытым слоем. Помогло ли это лучше различать данные?